In [1]:
# # Lab - 3 Taxi travel time

# Robin Jamsahar, Mohamed Osman

In [ ]:
#needed downloads
#pip install fastparquet
#pip install pyarrow

In [2]:
# Importing the libraries
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import datetime

In [3]:
#pd.show_versions()
#ytd = pd.read_parquet("data/trips/yellow_tripdata_2019-01.parquet",engine="fastparquet")
ytd = pq.read_table("data/trips/yellow_tripdata_2019-01.parquet").to_pandas()
gtd = pq.read_table("data/trips/green_tripdata_2019-01.parquet").to_pandas()
fhvtd = pq.read_table("data/trips/fhv_tripdata_2019-01.parquet").to_pandas()

FileNotFoundError: data/trips/yellow_tripdata_2019-01.parquet

In [ ]:
ytd.head()

We only need to keep columns tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, PULocationID and DOLocationID

In [ ]:
ytd = ytd[ytd["fare_amount"]>2.5]
gtd = gtd[gtd["fare_amount"]>2.5]
fhvtd = fhvtd[fhvtd["fare_amount"]>2.5]

In [ ]:
ytd = ytd[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
gtd = gtd[['lpep_pickup_datetime','lpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
fhvtd = fhvtd[['pickup_datetime','dropoff_datetime','PULocationID','DOLocationID']]

ytd.head()

To resume the data cleaning, we can convert the pickup and dropoff column to datatime. We can also remove rows where the pickuptime is greater than or equal to the dropoff time. We can also add a column 'trip_time' that says how long the trip lasted in seconds.

In [ ]:
ytd['tpep_pickup_datetime'] = pd.to_datetime(ytd['tpep_pickup_datetime'])
ytd['tpep_dropoff_datetime'] = pd.to_datetime(ytd['tpep_dropoff_datetime'])

ytd = ytd.drop(ytd[ytd['tpep_pickup_datetime'] >= ytd['tpep_dropoff_datetime']].index)
ytd['trip_time'] = (ytd['tpep_dropoff_datetime']-ytd['tpep_pickup_datetime']).dt.total_seconds()
ytd['speed'] = (ytd[''])

In [ ]:
ytd.head()

For convenience sake we can rename the pickup and dropoff columns to 'pickup_time' and 'dropoff_time' respectively. We can also only use time value in total seconds to make it easier to use in the model later.

In [ ]:
ytd['tpep_pickup_datetime'] = (ytd['tpep_pickup_datetime']-ytd['tpep_pickup_datetime'].dt.normalize()).dt.total_seconds()
ytd['tpep_dropoff_datetime'] = (ytd['tpep_dropoff_datetime']-ytd['tpep_dropoff_datetime'].dt.normalize()).dt.total_seconds()
ytd.rename(columns = {'tpep_pickup_datetime':'pickup_time','tpep_dropoff_datetime':'dropoff_time'},inplace = True)
ytd.info()
ytd.describe()

In [ ]:
ytd = ytd[ytd.PULocationID < 264]
ytd = ytd[ytd.DOLocationID < 264]
ytd = ytd[ytd.trip_time != 0]
ytd = ytd[ytd.trip_distance != 0.0]
ytd=ytd[ytd.trip_time<20000]
ytd.info()
ytd.describe()

We can now check if there are any missing values in our dataset.

In [ ]:
def missing_cols(df):
    '''prints out columns with its amount of missing values'''
    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        total += missing_vals
        pct = df[col].isna().mean() * 100
        if missing_vals != 0:
            print(f"{col} => {df[col].isnull().sum()},{round(pct,2)}%")

    if total == 0:
        print("no missing values left")

missing_cols(ytd)

In [ ]:
ytd.describe()
corr = ytd["trip_time"].corr(ytd["trip_distance"])
print(corr)

In [ ]:
from sklearn.model_selection import train_test_split

copy = ytd.copy()
copy["pickup_time"] = (ytd[("pickup_time")] - ytd[("pickup_time")].mean()) / ytd[("pickup_time")].std()
copy["trip_distance"] = (ytd[("trip_distance")] - ytd[("trip_distance")].mean()) / ytd[("trip_distance")].std()
copy["PULocationID"] = (ytd[("PULocationID")] - ytd[("PULocationID")].mean()) / ytd[("PULocationID")].std()
copy["DOLocationID"] = (ytd[("DOLocationID")] - ytd[("DOLocationID")].mean()) / ytd[("DOLocationID")].std()


#code = diabetes["Code"]
copy["trip_time"] = (ytd[("trip_time")] - ytd[("trip_time")].mean()) / ytd[("trip_time")].std()

X = np.asarray(copy[["pickup_time","trip_distance","PULocationID","DOLocationID"]])
Y = np.asarray(copy["trip_time"])

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.33,random_state = 1)

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm = lm.fit(X_train, Y_train)
Y_hat = lm.predict(X_test)
print("Linear Regression:")
print("Accuracy: ",r2_score(Y_test, Y_hat))
print("Error: ",mean_squared_error(Y_test, Y_hat))

from sklearn.ensemble import RandomForestRegressor

# f = RandomForestRegressor()
# f = f.fit(X_train, Y_train)
# Y_hat = t.predict(X_test)
# print("Random Forest Regressor:")
# print("Accuracy: ",r2_score(Y_test, Y_hat))
# print("Error: ",mean_squared_error(Y_test, Y_hat))



# X = ytd[['trip_distance','pickup_time']]
# Y = ytd['trip_time']
# X = X.values.reshape(len(X), 2)
# Y = Y.values.reshape(len(Y), 1)

# X_train = X[:int(0.8*len(X))]
# X_test = X[int(0.8*len(X)):]

# Y_train = Y[:int(0.8*len(Y))]
# Y_test = Y[int(0.8*len(Y)):]

# from sklearn.tree import DecisionTreeRegressor

# t = DecisionTreeRegressor()
# t = t.fit(X_train, Y_train)
# Y_hat = t.predict(X_test)
# print(r2_score(Y_test, Y_hat))

# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.linear_model import LinearRegression
# lm = LinearRegression()
# lm = lm.fit(X_train, Y_train)
# Y_hat = lm.predict(X_test)
# print(r2_score(Y_test, Y_hat))
# print(mean_squared_error(Y_test, Y_hat))